# DiCE Hyperparameter Results

In [7]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../..'))

%load_ext autoreload
%autoreload 2

import joblib
import pandas as pd
import numpy as np
from sklearn import neighbors
from sklearn import model_selection

import matplotlib.pyplot as plt
import seaborn as sns
from models import model_interface, model_loader, model_constants
from data import data_loader
from data.adapters import continuous_adapter
from scripts import fit_kde

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
RECOURSE_METHOD = 'dice'
RESULTS_DIR = '../../experiment_results/dice/dice_hyperparam'

# Preliminaries -- load everything

In [3]:
DATASET, DATASET_INFO = data_loader.load_data(data_loader.DatasetName('credit_card_default'), split="train")
MODEL = model_loader.load_model(model_constants.ModelType('logistic_regression'), data_loader.DatasetName('credit_card_default'))
ADAPTER = continuous_adapter.StandardizingAdapter(
    label_column = DATASET_INFO.label_column, positive_label=DATASET_INFO.positive_label
).fit(DATASET)

DROP_COLUMNS = ['step_id', 'path_id', 'run_id', 'batch_id']  # columns which are convenient to drop from the path_df

config_df = pd.read_csv(os.path.join(RESULTS_DIR, 'experiment_config_df.csv'))
path_df = pd.read_csv(os.path.join(RESULTS_DIR, f'{RECOURSE_METHOD}_paths_df.csv'))
config_df

,batch_id,run_id,run_seed,confidence_cutoff,dataset_name,max_iterations,model_type,noise_ratio,num_paths,rescale_ratio,split,elapsed_recourse_seconds
0,4,149,227,0.6,credit_card_default,50,logistic_regression,NaN,5,NaN,val,1.444696
1,18,550,3363,0.9,credit_card_default,50,logistic_regression,NaN,4,NaN,val,1.243439
2,1,53,211,0.6,credit_card_default,50,logistic_regression,NaN,2,NaN,val,0.884133
3,17,520,3363,0.9,credit_card_default,50,logistic_regression,NaN,3,NaN,val,1.072941
4,7,225,3439,0.7,credit_card_default,50,logistic_regression,NaN,3,NaN,val,1.062886
...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,61,3183,0.6,credit_card_default,50,logistic_regression,NaN,3,NaN,val,0.973757
596,14,437,1573,0.8,credit_card_default,50,logistic_regression,NaN,5,NaN,val,1.289615
597,11,335,8286,0.8,credit_card_default,50,logistic_regression,NaN,2,NaN,val,0.805957
598,8,249,1498,0.7,credit_card_default,50,logistic_regression,NaN,4,NaN,val,1.129014


## Load or Fit a KDE

In [8]:
KDE_DIRECTORY = '../../saved_models/kde/credit_card_default_kde.joblib'

if os.path.exists(KDE_DIRECTORY):
    KDE = joblib.load(KDE_DIRECTORY)
else:
    KDE = fit_kde.fit_kde('credit_card_default', KDE_DIRECTORY)

# Analyze the results

In [10]:
SPARSITY_EPSILON = 1e-5

def get_poi_cfes(path_df: pd.DataFrame):
    """Isolate the POIs (Points of Interest) and CFEs (Counterfactual Examples) from the full path results.
    
    POIs and CFEs are listed in the order they originally appear in. There is one POI and one CFE
    for every path that appears in the DataFrame."""
    pathscopy = path_df.copy()
    pathscopy['next_step_id'] = 0
    pathscopy.loc[:,'next_step_id'].iloc[0:-1] = pathscopy.loc[:,'step_id'].iloc[1:]
    cfes = pathscopy[pathscopy.step_id >= pathscopy.next_step_id].drop(columns='next_step_id')
    return pathscopy[pathscopy.step_id == 0].drop(columns='next_step_id'), cfes

def get_sparsity(path: pd.DataFrame):
    """Returns the maximum number of features changed in any single iteration
    along the path."""
    if path.shape[0] == 1:
        return np.nan
    path_sparsity = np.zeros(path.shape[0])
    for i in range(1, path.shape[0]):
        path_sparsity[i] = (np.abs(path.iloc[i] - path.iloc[i - 1]) > SPARSITY_EPSILON).sum()
    return np.max(path_sparsity)

def get_path_length(path: pd.DataFrame):
    """Returns the sum of euclidean distances along the path."""
    total = 0
    for i in range(1, path.shape[0]):
        total += np.linalg.norm(path.iloc[i] - path.iloc[i - 1])
    if total == 0:
        return np.nan
    return total

def get_cfe_distance(path: pd.DataFrame):
    """Returns the euclidean distance between the first and last points in the path."""
    if len(path) == 1:
        return np.nan
    return np.linalg.norm(path.iloc[-1] - path.iloc[0])


def analyze_paths(paths: pd.DataFrame, poi_kdes, cfe_kdes, cfe_probs, config_df):
    """Returns a DataFrame containing per-path results.
    
    Each row corresponds to a specific path. Each column is a result metric.
    
    Args:
        paths: The path_df DataFrame to analyze.
        poi_kdes: The KDE scores for the POIs.
        cfe_kdes: The KDE scores for the CFEs.
        config_df: The experiment_config_df for the experiment."""
    columns = ['run_id', 'path_id', 'success', 'proximity', 'path_length',
               'iteration_count', 'poi_density', 'cfe_density', 
               'actual_sparsity']
    col_idx = {}
    for i, col in enumerate(columns):
        col_idx[col] = i

    results = np.zeros((len(poi_kdes), len(columns)))

    i = 0
    for run_id in paths.run_id.unique():
        run_paths = paths[paths.run_id == run_id]
        for path_id in run_paths.path_id.unique():
            path = ADAPTER.transform(run_paths[run_paths.path_id == path_id].drop(columns=DROP_COLUMNS))
            results[i,col_idx['run_id']] = run_id
            results[i,col_idx['path_id']] = path_id

            desired_proba = config_df[config_df.run_id == run_id].confidence_cutoff.iloc[0]
            actual_proba = cfe_probs[i]

            results[i,col_idx['success']] = 1 if actual_proba >= desired_proba else 0
            results[i,col_idx['path_length']] = get_path_length(path)
            results[i,col_idx['iteration_count']] = len(path)
            results[i,col_idx['proximity']] = get_cfe_distance(path)
            results[i,col_idx['poi_density']] = poi_kdes[i]
            results[i,col_idx['cfe_density']] = cfe_kdes[i]
            results[i,col_idx['actual_sparsity']] = get_sparsity(path)
            i += 1

    return pd.DataFrame(data=results, columns=columns)

pois, cfes = get_poi_cfes(path_df)
poi_kdes = KDE.score_samples(ADAPTER.transform(pois.drop(columns=DROP_COLUMNS)))
cfe_kdes = KDE.score_samples(ADAPTER.transform(cfes.drop(columns=DROP_COLUMNS)))
cfe_probs = MODEL.predict_pos_proba(cfes.drop(columns=DROP_COLUMNS)).to_numpy()

results = analyze_paths(path_df, poi_kdes, cfe_kdes, cfe_probs, config_df)
results = results.merge(config_df, how='left', on='run_id').drop(
    columns=['dataset_name', 'max_iterations', 'model_type', 'noise_ratio',
             'rescale_ratio', 'run_seed', 'split'])  # uninteresting columns
results

/tmp/ipykernel_1563/3412013495.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pathscopy.loc[:,'next_step_id'].iloc[0:-1] = pathscopy.loc[:,'step_id'].iloc[1:]


,run_id,path_id,success,proximity,path_length,iteration_count,poi_density,cfe_density,actual_sparsity,batch_id,confidence_cutoff,num_paths,elapsed_recourse_seconds
0,149.0,0.0,1.0,27.201860,27.201860,2.0,-1.874542,-1319.776594,1.0,4,0.6,5,1.444696
1,149.0,1.0,1.0,39.897809,39.897809,2.0,-1.874542,-1819.347564,1.0,4,0.6,5,1.444696
2,149.0,2.0,1.0,45.312073,45.312073,2.0,-1.874542,-7709.476535,2.0,4,0.6,5,1.444696
3,149.0,3.0,1.0,23.008441,23.008441,2.0,-1.874542,-1263.065058,2.0,4,0.6,5,1.444696
4,149.0,4.0,1.0,11.496382,11.496382,2.0,-1.874542,-28.483340,2.0,4,0.6,5,1.444696
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,249.0,3.0,1.0,34.217198,34.217198,2.0,-2.399717,-1843.717517,1.0,8,0.7,4,1.129014
1796,90.0,0.0,1.0,32.613513,32.613513,2.0,-21.102418,-1822.465708,2.0,3,0.6,4,1.126958
1797,90.0,1.0,1.0,67.313789,67.313789,2.0,-21.102418,-7781.350053,1.0,3,0.6,4,1.126958
1798,90.0,2.0,1.0,35.450984,35.450984,2.0,-21.102418,-2578.075539,1.0,3,0.6,4,1.126958


# Choosing metrics

Can we just use the metrics chosen by StEP?
* num_paths=3
* confidence_cutoff=0.7

Let's see what the best-performing parameter settings look like.

In [11]:
DROP_METRICS = ['run_id', 'elapsed_recourse_seconds', 'negative_cfe_density',
                'path_id', 'batch_id', 'actual_sparsity']

results['negative_cfe_density'] = -results['cfe_density']
results.groupby('batch_id', as_index=False).mean().sort_values(
    ['success', 'path_length', 'proximity', 'negative_cfe_density', 'iteration_count']).iloc[:5].drop(
        columns=DROP_METRICS)

,success,proximity,path_length,iteration_count,poi_density,cfe_density,confidence_cutoff,num_paths
10,1.0,35.054767,35.054767,2.0,-7.499876,-2296.312941,0.8,1.0
6,1.0,35.096680,35.096680,2.0,-7.499876,-2883.638705,0.7,2.0
2,1.0,35.656831,35.656831,2.0,-7.499876,-3728.503310,0.6,3.0
8,1.0,35.955748,35.955748,2.0,-7.499876,-3159.245011,0.7,4.0
1,1.0,36.418244,36.418244,2.0,-7.499876,-3764.884663,0.6,2.0


Takeaways

* DiCE is not very sensitive to its hyperparameters
* It is most sensitive in cfe_density, which mostly depends on num_paths
* DiCE hyperparameter response resembles StEP response

For these reasons, it is fair to use the same parameters as StEP.

## Impact of confidence_cutoff

In [12]:
results.groupby('confidence_cutoff').mean().drop(columns=DROP_METRICS)

,success,proximity,path_length,iteration_count,poi_density,cfe_density,num_paths
confidence_cutoff,,,,,,,
0.6,1.0,37.005525,37.005525,2.0,-7.499876,-3890.401425,3.666667
0.7,1.0,36.412764,36.412764,2.0,-7.499876,-3234.198810,3.666667
0.8,1.0,38.840406,38.840406,2.0,-7.499876,-3445.152007,3.666667
0.9,1.0,41.348722,41.348722,2.0,-7.499876,-4088.438633,3.666667


## Impact of num_paths

In [13]:
results.groupby('num_paths').mean().drop(columns=DROP_METRICS)

,success,proximity,path_length,iteration_count,poi_density,cfe_density,confidence_cutoff
num_paths,,,,,,,
1,1.0,38.488328,38.488328,2.0,-7.499876,-3463.175445,0.75
2,1.0,38.045591,38.045591,2.0,-7.499876,-3517.397934,0.75
3,1.0,38.118294,38.118294,2.0,-7.499876,-3584.985473,0.75
4,1.0,38.402391,38.402391,2.0,-7.499876,-3694.360335,0.75
5,1.0,38.696772,38.696772,2.0,-7.499876,-3787.569343,0.75
